In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
%matplotlib inline
import math
from keras.utils import np_utils

In [ ]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (test_x, test_y) = cifar10.load_data()

num_classes = 10
train_y_cls = np_utils.to_categorical(y_train,num_classes)
test_y_cls = np_utils.to_categorical(test_y,num_classes)

print("Size of:")
print("- Training-set:\t\t{}".format(len(x_train)))
print("- Test-set:\t\t{}".format(len(test_x)))

In [ ]:
def load_label_names():
    return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

class_names = load_label_names()

def label_to_array(cls_true):
    true = cls_true.tolist()
    true_label = []
    for l in true:
        true_label += l
    return true_label
    

### Create Random Validation and train data

In [ ]:
combined_size = x_train.shape[0]
split_size = 0.2

In [ ]:
# Helper-function for splitting the combined data-set into a random training- and validation-set.

def random_training_set(split_size):
    
    train_size = int(combined_size*(1-split_size))

    # Create a randomized index into the full / combined training-set.
    idx = np.random.permutation(combined_size)

    # Split the random index into training- and validation-sets.
    idx_train = idx[0:train_size]
    idx_validation = idx[train_size:]

    # Select the images and labels for the new training-set.
    train = x_train[idx_train, :]
    labels = y_train[idx_train, :]

    # Select the images and labels for the new validation-set.
    x_validation = x_train[idx_validation, :]
    y_validation = y_train[idx_validation, :]

    # Return the new training- and validation-sets.
    return train, labels, x_validation, y_validation


In [ ]:
# Split data into random training and validation data
train_x, train_y, validation_x, validation_y = random_training_set(split_size)

train_y_cls = np_utils.to_categorical(train_y,num_classes)
validation_y_cls = np_utils.to_categorical(validation_y,num_classes)

print (train_x.shape)
print (validation_x.shape)

In [ ]:
#Data Dimensions

# CIFAR-10 images are 32 pixels in each dimension.
img_size = 32

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images:
num_channels = 3

# Number of classes, one class for each of 10 digits.
num_classes = 10

input_shape = (img_size, img_size, num_channels)

In [ ]:
#Plotting Images

def plot_images(images, cls_true, cls_pred=None, smooth=True):

    # Create figure with sub-plots.
    fig, axes = plt.subplots(3, 3)

    # Adjust vertical spacing if we need to print ensemble and best-net.
    if cls_pred is None:
        hspace = 0.3
    else:
        hspace = 0.6
    fig.subplots_adjust(hspace=hspace, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Interpolation type.
        if smooth:
            interpolation = 'spline16'
        else:
            interpolation = 'nearest'

        # Plot image.
        ax.imshow(images[i, :, :, :],
                  interpolation=interpolation)
            
        # Name of the true class.
        cls_true_name = class_names[cls_true[i]]

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true_name)
        else:
            # Name of the predicted class.
            cls_pred_name = class_names[cls_pred[i]]

            xlabel = "True: {0}\nPred: {1}".format(cls_true_name, cls_pred_name)

        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [ ]:
#Preparing Data
def data_prep(train, validation,test, subtract_pixel_mean):
    # Normalize Data
    train = train.astype('float32')/255
    validation = validation.astype('float32')/255
    test = test.astype('float32')/255
    
    # If subtract pixel mean is enabled
    if subtract_pixel_mean:
        train_mean = np.mean(train, axis=0)
        train -= train_mean
        validation -= train_mean
        test -= train_mean
        
    return train, validation, test


In [ ]:
# Subtract per pixel
subtract_pixel_mean = True

train_x, validation_x, test_x = data_prep(train_x, validation_x, test_x, subtract_pixel_mean)

## Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

def augment_batches(batch_size, data):
    '''
    Return a total of `num` samples
    '''
    
    images=batch_size
    while(images + batch_size <= len(data)):
        prev_no = images
        images += batch_size
        data[prev_no:images] = crop_images(data[prev_no:images])
    
    return data


# Padding images to 36x36 then randomly crop 32x32 size images
def crop_images(train):
    
    # Zero pad images to 36x36 size
    train = tf.image.resize_with_crop_or_pad(train,
                                             target_height=36,
                                             target_width=36).numpy()
    train = tf.image.random_crop(train, size=[train.shape[0], 32, 32, 3])
    
    return train.numpy()


In [ ]:
## Choose if images should be randomly cropped
crop_images = False

if (crop_images):
    train_x = augment_batches(train_x)


In [ ]:
# Get the first 9 images from the test-set.
images = train_x[0:9]

# Get the true classes for those images.
cls_true = train_y[0:9]
cls_true = label_to_array(cls_true)

# Plot the images and labels using our helper-function above.
plot_images(images, cls_true)


### Import Required Libraries

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D, Add
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import SGD

### Define an Inception Module

In [ ]:
def conv_module(net, filters, shape, stage, block):
    '''
    Arguments: 
    1. net -- input tensor of shape(m, h_prev, w_prev, c_prev)
    2. shape -- integer, specifying the shape of the middle CONV's
    window for the main path
    3. filters -- list of integers, defining the no of filters of 
    CONV layers in the main path
    4. stage -- integer, used to name the layers depending on their 
    position in the network
    5. block -- string/character used to name the layers depending
    on their position in the network
    
    Returns:
    net -- output of the identity block, tensor of shape
    (h_curr, w_curr, c_curr)
    '''
    
    # Defining name basis
    conv_name_base = 'res' + stage + block 
    bn_name_base = 'bn' + stage + block 
    
    # Conv component
    net = Conv2D(filters=filters, kernel_size=(shape,shape),
                 name = conv_name_base ,
                 strides=(1,1), padding='same')(net)
    net = BatchNormalization(axis = 3, name = bn_name_base)(net)
    net = Activation('relu')(net)
    
    return net

In [ ]:
def inception_module(net, 
                     filter1x1,
                     filter3x3_reduce,
                     filter3x3,
                     filter5x5_reduce,
                     filter5x5v1,
                     filter5x5v2,
                     filterpool,
                     stage):
        
    # 1x1 branch
    net_1x1 = conv_module(net, filter1x1, shape=1, stage=stage, block='1x1')
    
    # 3x3 branch
    net_3x3 = conv_module(net, filter3x3_reduce, shape=1, stage=stage, block='3x3_downsample')
    net_3x3 = conv_module(net_3x3, filter3x3, shape=3, stage=stage, block='3x3')
    
    # 5x5 branch (use 2 3x3 conv)
    net_3x3v1 = conv_module(net, filter5x5_reduce, shape=1, stage=stage, block='5x5_downsample')
    net_3x3v1 = conv_module(net_3x3v1, filter5x5v1, shape=3, stage=stage, block='3x3v1')
    net_3x3v2 = conv_module(net_3x3v1, filter5x5v2, shape=3, stage=stage, block='3x3v2')
    
    # MaxPool branch
    net_pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(net)
    net_pool = conv_module(net_pool, filterpool, shape=1, stage=stage, block='pool')
    
    # Concatinate all branches together
    net = concatenate([net_3x3, net_3x3v2, net_1x1, net_pool],
                     axis=3, name='inception'+stage )
    
    return net
    

In [ ]:
def inception(input_shape, num_classes):
    """
    Implementation of the ImageNet having the following architecture:
    
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define a tensor with input shape
    inputs = Input(input_shape)
    
    # Stage 1 --> 3x3 Convolution with 16 filters and stride 1
    net = Conv2D(32, (3,3), strides=(1,1), padding='same',
                name='conv1')(inputs)
    net = BatchNormalization(axis=3, name='bn_conv1')(net)
    net = Activation('relu')(net)
    
    # No of inception_modules 
    # Assume 6 modules (loss after each 2 modules)
    
    net = inception_module(net, 
                           filter1x1=32,
                           filter3x3_reduce=24,
                           filter3x3=32,
                           filter5x5_reduce=8,
                           filter5x5v1=16,
                           filter5x5v2=16,
                           filterpool=16,
                           stage='1a')
    
    # Loss after first inception module
    loss1 = AveragePooling2D((3,3),strides=(2,2),name='loss1')(net)
    loss1 = Conv2D(16,(1,1), padding='same',
                        activation='relu',
                        name='loss1-conv')(loss1)
    loss1 = Flatten()(loss1)
    loss1 = Dense(32,activation='relu',name='loss1-fc')(loss1)
    loss1 = Dropout(0.5)(loss1)
    loss1 = Dense(num_classes, name='loss1-classifier',)(loss1)
    loss1 = Activation('softmax')(loss1)
    
    # Continue the NN
    net = inception_module(net, 
                           filter1x1=20,
                           filter3x3_reduce=16,
                           filter3x3=32,
                           filter5x5_reduce=12,
                           filter5x5v1=16,
                           filter5x5v2=16,
                           filterpool=16,
                           stage='2a')
    
    # Loss after 2nd inception module
    loss2 = AveragePooling2D((3,3),strides=(2,2),name='loss2')(net)
    loss2 = Conv2D(16,(1,1), padding='same',
                        activation='relu',
                        name='loss2-conv')(loss2)
    loss2 = Flatten()(loss2)
    loss2 = Dense(32,activation='relu',name='loss2-fc')(loss2)
    loss2 = Dropout(0.5)(loss2)
    loss2 = Dense(num_classes, name='loss2-classifier')(loss2)
    loss2 = Activation('softmax')(loss2)
    
    # Continue the NN
    net = inception_module(net, 
                           filter1x1=32,
                           filter3x3_reduce=20,
                           filter3x3=32,
                           filter5x5_reduce=16,
                           filter5x5v1=32,
                           filter5x5v2=32,
                           filterpool=32,
                           stage='3a')
    
    # Loss of final module
    loss3 = AveragePooling2D((3, 3), strides=(2,2))(net)
    loss3 = Flatten()(loss3)
    loss3 = Dense(128, name='loss3_dense')(loss3)
    loss3 = Activation('relu')(loss3)
    loss3 = Dropout(0.4)(loss3)
    loss3 = Dense(num_classes, name='loss3-classifier')(loss3)
    loss3 = Activation("softmax")(loss3)
    
    model = Model(inputs=inputs, outputs=[loss1,loss2,loss3], name='Inception')
    
    return model
    

In [ ]:
# Creating Model
model = inception(input_shape, num_classes)

In [ ]:
num_epochs = 50

# Initialize the optimizer and compile the model
optimizer = SGD(lr=0.0, momentum=0.9, decay=0.0001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer,
             metrics=['accuracy'])


## Step Decay
By using step decay, we schedule drop the learning rate by a factor every few epochs.

#### Is useful when training for long epochs

In [ ]:
# Define step decay function
class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))
        print('lr:', step_decay(len(self.losses)))

def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.1
    epochs_drop = num_epochs/2
    lrate = initial_lrate * math.pow(drop, math.floor((epoch)/epochs_drop))
    return lrate

# learning schedule callback
loss_history = LossHistory()
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
callbacks_list = [loss_history, lrate]

In [ ]:
model.summary()

### Training the Model

In [ ]:
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    h = model.fit(train_x,
                  [train_y_cls,train_y_cls,train_y_cls],
                  batch_size = 128,
                  validation_data=(validation_x,
                                   [validation_y_cls,validation_y_cls,validation_y_cls]),
                  callbacks=callbacks_list, 
                  epochs = num_epochs,
                  verbose = 1)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train_x)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(train_x, [train_y_cls,train_y_cls,train_y_cls],
                                     batch_size=128),
                                     validation_data=(validation_x, 
                                                     [validation_y_cls,validation_y_cls,validation_y_cls]),
                                     epochs=num_epochs, verbose=1, workers=4,
                                     callbacks=callbacks_list)

In [ ]:
result = model.evaluate(test_x, test_y_cls)

In [ ]:
#Predict on test_set
pred_y = model.predict(test_x)
pred_cls = np.argmax(pred_y, axis=1)

In [ ]:
#Some predicted Images
# Get the first 9 images from the test-set.
images = test_x[0:9]

# Get the true classes for those images.
cls_true = test_y[0:9]
cls_true = label_to_array(cls_true)

#Get Predicted Classes
cls_pred = pred_cls[0:9]
cls_pred = cls_pred.tolist()

# Plot the images and labels using our helper-function above.
plot_images(images, cls_true, cls_pred)